In [0]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data
import random

In [2]:
df = pd.read_csv('/content/jenkins.csv')
df.head()

,fix,ns,nd,nf,entropy,la,ld,lt,ndev,age,nuc,exp,rexp,sexp,author_date_unix_timestamp,classification,contains_bug
0,False,7.0,7.0,7.0,2.641604,9.0,9.0,426.428571,100.0,0.000093,1.0,5171.0,30.227271,1472.714286,1555326371,None,False
1,False,7.0,7.0,7.0,2.750000,8.0,8.0,426.428571,100.0,6.314775,2.0,5170.0,29.227271,1471.714286,1555326363,None,False
2,False,1.0,1.0,2.0,0.906580,15.0,44.0,96.000000,4.0,0.034722,2.0,629.0,14.828373,414.000000,1554971763,None,False
3,False,1.0,1.0,1.0,0.000000,0.0,0.0,40.000000,1.0,0.000012,1.0,4.0,3.058824,3.000000,1554969774,None,False
4,False,1.0,2.0,4.0,1.662506,14.0,10.0,67.000000,6.0,21.280683,4.0,3.0,2.058824,2.000000,1554967752,Feature Addition,False


In [3]:
df = df.sample(frac=1)
label_col = 'contains_bug'
features_cols = ['fix', 'ns', 'nd', 'nf', 'entropy', 'la', 'ld', 'lt', 'ndev', 'age', 'nuc', 'exp', 'rexp', 'sexp', 'classification']
x = df[features_cols]
x['fix'] = x['fix'].astype('int')
df_classification = pd.get_dummies(x, columns=['classification'])
x = pd.concat([x, df_classification], axis='columns')
x = x.drop(['classification'], axis='columns')
x = x.values
y = df[label_col]
y = y.astype('category')
y = y.cat.codes
y = y.values

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [4]:
print(x[:3])
print(y[:3])

[[0.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  0.00000000e+00 1.00000000e+00 1.00000000e+00 9.30000000e+02
  5.00000000e+00 6.95868056e-01 1.00000000e+00 3.78000000e+02
  4.38635226e+01 4.30000000e+01 0.00000000e+00 1.00000000e+00
  1.00000000e+00 1.00000000e+00 0.00000000e+00 1.00000000e+00
  1.00000000e+00 9.30000000e+02 5.00000000e+00 6.95868056e-01
  1.00000000e+00 3.78000000e+02 4.38635226e+01 4.30000000e+01
  0.00000000e+00 0.00000000e+00 0.00000000e+00 1.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  0.00000000e+00 1.00000000e+00 9.00000000e+00 9.32000000e+02
  3.00000000e+01 9.60648148e-03 1.00000000e+00 3.10500000e+03
  9.66193307e+01 1.85400000e+03 0.00000000e+00 1.00000000e+00
  1.00000000e+00 1.00000000e+00 0.00000000e+00 1.00000000e+00
  9.00000000e+00 9.32000000e+02 3.00000000e+01 9.60648148e-03
  1.00000000e+00 3.10500000e+03 9.66193307e+01 1.85400000e+03
  0.00000000e+00 0.00000000e+00 0.000

In [0]:
val_index = int( len(x) * 0.8 )
test_index = int( len(x) * 0.9 )
x_train, y_train = x[:val_index], y[:val_index]
x_val, y_val = x[val_index:test_index], y[val_index:test_index]
x_test, y_test = x[test_index:], y[test_index:]

In [6]:
print(x_train[:3])
print(x_val[:3])
print(x_test[:3])

[[0.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  0.00000000e+00 1.00000000e+00 1.00000000e+00 9.30000000e+02
  5.00000000e+00 6.95868056e-01 1.00000000e+00 3.78000000e+02
  4.38635226e+01 4.30000000e+01 0.00000000e+00 1.00000000e+00
  1.00000000e+00 1.00000000e+00 0.00000000e+00 1.00000000e+00
  1.00000000e+00 9.30000000e+02 5.00000000e+00 6.95868056e-01
  1.00000000e+00 3.78000000e+02 4.38635226e+01 4.30000000e+01
  0.00000000e+00 0.00000000e+00 0.00000000e+00 1.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  0.00000000e+00 1.00000000e+00 9.00000000e+00 9.32000000e+02
  3.00000000e+01 9.60648148e-03 1.00000000e+00 3.10500000e+03
  9.66193307e+01 1.85400000e+03 0.00000000e+00 1.00000000e+00
  1.00000000e+00 1.00000000e+00 0.00000000e+00 1.00000000e+00
  9.00000000e+00 9.32000000e+02 3.00000000e+01 9.60648148e-03
  1.00000000e+00 3.10500000e+03 9.66193307e+01 1.85400000e+03
  0.00000000e+00 0.00000000e+00 0.000

In [0]:
mean_train = x_train.mean(axis=0)
std_train = x_train.std(axis=0)

In [8]:
print(mean_train)
print(std_train)

[2.77527799e-01 1.58919059e+00 2.51073183e+00 3.97667442e+00
 7.31179694e-01 5.32847168e+01 2.48240497e+01 7.08735227e+02
 1.90427205e+01 8.18745511e+01 1.91936902e+00 1.74538242e+03
 8.58253478e+01 9.03776002e+02 2.77527799e-01 1.58919059e+00
 2.51073183e+00 3.97667442e+00 7.31179694e-01 5.32847168e+01
 2.48240497e+01 7.08735227e+02 1.90427205e+01 8.18745511e+01
 1.91936902e+00 1.74538242e+03 8.58253478e+01 9.03776002e+02
 2.77527799e-01 1.55521076e-01 3.68761314e-02 4.79751746e-01
 1.53090251e-02 3.50142229e-02]
[4.47779097e-01 1.60848444e+00 6.43543463e+00 2.48125619e+01
 1.11612212e+00 6.63681017e+02 5.16477345e+02 1.06804000e+03
 2.69330566e+01 2.14149149e+02 7.36228375e+00 1.98154753e+03
 8.22352742e+01 1.31067084e+03 4.47779097e-01 1.60848444e+00
 6.43543463e+00 2.48125619e+01 1.11612212e+00 6.63681017e+02
 5.16477345e+02 1.06804000e+03 2.69330566e+01 2.14149149e+02
 7.36228375e+00 1.98154753e+03 8.22352742e+01 1.31067084e+03
 4.47779097e-01 3.62400705e-01 1.88457641e-01 4.99589

In [0]:
x_train = (x_train - mean_train) / std_train
x_val = (x_val - mean_train) / std_train
x_test = (x_test - mean_train) / std_train

In [10]:
print(x_train[:3])
print(x_val[:3])
print(x_test[:3])

[[-0.6197873  -0.3663017  -0.23475211 -0.11996643 -0.65510725 -0.07877989
  -0.04612797  0.207169   -0.52139349 -0.37907544 -0.12487552 -0.69005784
  -0.51026552 -0.65674461 -0.6197873  -0.3663017  -0.23475211 -0.11996643
  -0.65510725 -0.07877989 -0.04612797  0.207169   -0.52139349 -0.37907544
  -0.12487552 -0.69005784 -0.51026552 -0.65674461 -0.6197873  -0.42914121
  -0.19567332  1.04135075 -0.12468775 -0.19048544]
 [-0.6197873  -0.3663017  -0.23475211 -0.11996643 -0.65510725 -0.07877989
  -0.03063842  0.20904158  0.40683387 -0.38228004 -0.12487552  0.68613927
   0.13125734  0.72499057 -0.6197873  -0.3663017  -0.23475211 -0.11996643
  -0.65510725 -0.07877989 -0.03063842  0.20904158  0.40683387 -0.38228004
  -0.12487552  0.68613927  0.13125734  0.72499057 -0.6197873  -0.42914121
  -0.19567332  1.04135075 -0.12468775 -0.19048544]
 [ 1.61345674 -0.3663017  -0.23475211 -0.11996643 -0.65510725 -0.05617867
  -0.03257461 -0.6139613  -0.55852259  0.52427455 -0.12487552 -0.51191425
  -0.35664

In [0]:
x_train, y_train = torch.from_numpy(x_train), torch.from_numpy(y_train)
x_val, y_val = torch.from_numpy(x_val), torch.from_numpy(y_val)
x_test, y_test = torch.from_numpy(x_test), torch.from_numpy(y_test)

In [0]:
train_dataset = data.TensorDataset(x_train, y_train)
val_dataset = data.TensorDataset(x_val, y_val)
test_dataset = data.TensorDataset(x_test, y_test)

train_dataloader = data.DataLoader(train_dataset, batch_size=512, shuffle=True)
val_dataloader = data.DataLoader(val_dataset, batch_size=32)
test_dataloader = data.DataLoader(test_dataset, batch_size=32)

In [13]:
inputs, targets = next(iter(train_dataloader))
print(inputs[:3])
print(targets[:3])

inputs, targets = next(iter(val_dataloader))
print(inputs[:3])
print(targets[:3])

inputs, targets = next(iter(test_dataloader))
print(inputs[:3])
print(targets[:3])

tensor([[ 1.6135e+00,  8.7710e-01,  7.6027e-02, -3.9362e-02,  1.8595e-03,
          1.3132e-02, -4.6128e-02,  3.2577e-01, -1.1297e-01,  1.1914e-01,
          1.4678e-01, -8.7880e-01, -1.0112e+00, -6.8879e-01,  1.6135e+00,
          8.7710e-01,  7.6027e-02, -3.9362e-02,  1.8595e-03,  1.3132e-02,
         -4.6128e-02,  3.2577e-01, -1.1297e-01,  1.1914e-01,  1.4678e-01,
         -8.7880e-01, -1.0112e+00, -6.8879e-01,  1.6135e+00, -4.2914e-01,
         -1.9567e-01, -9.6029e-01, -1.2469e-01, -1.9049e-01],
        [-6.1979e-01, -3.6630e-01, -7.9362e-02, -7.9664e-02,  2.4032e-01,
         -3.0564e-02,  9.1342e-02, -3.7427e-01, -3.7288e-01, -3.5514e-01,
         -1.2488e-01, -8.7779e-01, -1.0285e+00, -6.8879e-01, -6.1979e-01,
         -3.6630e-01, -7.9362e-02, -7.9664e-02,  2.4032e-01, -3.0564e-02,
          9.1342e-02, -3.7427e-01, -3.7288e-01, -3.5514e-01, -1.2488e-01,
         -8.7779e-01, -1.0285e+00, -6.8879e-01, -6.1979e-01,  2.3302e+00,
         -1.9567e-01, -9.6029e-01, -1.2469e-01, -1

In [0]:
class Classifier(nn.Module):
  def __init__(self, epochs, input_size, hidden_size, val_loss=None):
    super(Classifier, self).__init__()
    self.epochs = epochs
    self.input_size = input_size
    self.hidden_size = hidden_size
    self.val_loss = val_loss
    self.fc1 = nn.Linear(input_size, hidden_size)
    self.fc2 = nn.Linear(hidden_size, hidden_size)
    self.fcout = nn.Linear(hidden_size, 1)
  
  def forward(self, x):
    x = torch.sigmoid(self.fc1(x))
    x = torch.sigmoid(self.fc2(x))
    x = torch.sigmoid(self.fcout(x))
    return x

In [15]:
classifier = Classifier(epochs=100, input_size=x.shape[1], hidden_size=256)
classifier

Classifier(
  (fc1): Linear(in_features=34, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=256, bias=True)
  (fcout): Linear(in_features=256, out_features=1, bias=True)
)

In [0]:
filename = 'classifier.cpt'
def save_classifier(classifier):
  checkpoint = {
      'epochs': classifier.epochs,
      'input_size': classifier.input_size,
      'hidden_size': classifier.hidden_size,
      'val_loss': classifier.val_loss,
      'state_dict': classifier.state_dict()
  }
  with open(filename, 'wb') as f:
    torch.save(checkpoint, f)

def load_classifier():
  with open(filename, 'rb') as f:
    checkpoint = torch.load(f)
  classifier = Classifier(checkpoint['epochs'], checkpoint['input_size'], checkpoint['hidden_size'], checkpoint['val_loss'])
  classifier.load_state_dict(checkpoint['state_dict'])
  return classifier

In [0]:
save_classifier(classifier)
classifier = load_classifier()

In [18]:
if torch.cuda.is_available():
  classifier = classifier.cuda()
  inputs = inputs.cuda()

classifier(inputs[:3].float())

tensor([[0.4838],
        [0.4883],
        [0.4868]], device='cuda:0', grad_fn=<SigmoidBackward>)

In [0]:
criterion = nn.BCELoss()
optimizer = optim.Adam(params=classifier.parameters(), lr=0.03)

In [20]:
train_loss = 0
epochs = classifier.epochs
for epoch in range(epochs):
  classifier.train()
  for inputs, targets in train_dataloader:
    if torch.cuda.is_available():
      inputs, targets = inputs.cuda(), targets.cuda()

    outputs = classifier(inputs.float())
    loss = criterion(outputs.squeeze(), targets.float())
    train_loss += loss.item()

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
  
  with torch.no_grad():
    classifier.eval()
    val_loss = 0
    for inputs, targets in val_dataloader:
      if torch.cuda.is_available():
        inputs, targets = inputs.cuda(), targets.cuda()

      outputs = classifier(inputs.float())
      loss = criterion(outputs.squeeze(), targets.float())
      val_loss += loss.item()

    train_loss = train_loss / len(train_dataloader)
    val_loss = val_loss / len(val_dataloader)
    print('Epoch: {}, Train loss: {}, Val loss: {}'.format(epoch, train_loss, val_loss))

    if classifier.val_loss is None or val_loss < classifier.val_loss:
      classifier.val_loss = val_loss
      classifier.epochs = epoch
      save_classifier(classifier)


Epoch: 0, Train loss: 0.6348657419807032, Val loss: 0.5236511626525929
Epoch: 1, Train loss: 0.5192720610596797, Val loss: 0.49649959214423833
Epoch: 2, Train loss: 0.48539842748510753, Val loss: 0.48249259355821106
Epoch: 3, Train loss: 0.4587922500371588, Val loss: 0.4401989179222207
Epoch: 4, Train loss: 0.428354355344646, Val loss: 0.42181926103014694
Epoch: 5, Train loss: 0.41587551101910103, Val loss: 0.41979212686419487
Epoch: 6, Train loss: 0.41315477737859335, Val loss: 0.411880879613914
Epoch: 7, Train loss: 0.4045073530258514, Val loss: 0.40399081103111567
Epoch: 8, Train loss: 0.40109423388823545, Val loss: 0.3995685163689287
Epoch: 9, Train loss: 0.4016550961266967, Val loss: 0.4062763591738124
Epoch: 10, Train loss: 0.3971655750042801, Val loss: 0.40408140067991455
Epoch: 11, Train loss: 0.39365218209531905, Val loss: 0.3954143302613183
Epoch: 12, Train loss: 0.3893046303609241, Val loss: 0.4061592317333347
Epoch: 13, Train loss: 0.39273722136234823, Val loss: 0.391820294

In [0]:
def accuracy(classifier, dataloader):
  with torch.no_grad():
    classifier.eval()
    accuracy = 0
    for inputs, targets in dataloader:
      if torch.cuda.is_available():
        inputs, targets = inputs.cuda(), targets.cuda()
      
      outputs = classifier(inputs.float())
      predictions = torch.round(outputs).int()
      equals = (targets == predictions.squeeze())
      accuracy += torch.mean(equals.float())

    accuracy = accuracy / len(dataloader)
    return accuracy

In [22]:
classifier = load_classifier()
if torch.cuda.is_available():
  classifier = classifier.cuda()
train_accuracy = accuracy(classifier, train_dataloader)
test_accuracy = accuracy(classifier, test_dataloader)
print('Epochs: {}'.format(classifier.epochs))
print('Train accuracy: {}'.format(train_accuracy.item()))
print('Test accuracy: {}'.format(test_accuracy.item()))

Epochs: 32
Train accuracy: 0.8506277203559875
Test accuracy: 0.8401461243629456
